# 默认代码

In [ ]:
# 导入形式（必须是双引号）

# %%capture
# %run "viz_Tool.ipynb"

# 最好同时也导入 file_Tool.ipynb 文件，因为下面的绘图方法中会调用到 file_Tool.ipynb 中的数据处理方法

In [ ]:
import pyecharts.options as opts
from pyecharts.charts import Line
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK

# 折线图
>- 具体语法见 `viz_Use.ipynb` 作图规范

In [ ]:
# X 、Y 可以是 List 、ndarray 、DataFrame 的两列 Series
# ignnore 表示是否将 Y 数据中的 0.0 转化为 None
#         以获得更加平滑的折线图，而不是忽上忽下的尖山图
#         但是两种方法的均值计算不同，需要注意
# Y_name 为 Y 数据图例名称
# Y_unit 为 Y 数据单位


def draw_Line(X , Y , ignore = False , Y_name = '' , Y_unit = ''):
    
    # 预处理
    X = list(X)
    Y = list(Y)
    X = [str(x) for x in X]
    
    # 是否将 0 或 0.0 转化为 None
    if ignore:
        Y = trans_Y_for_Viz(Y)      # file_Tool.ipynb 中的数据处理方法
    
    if len(X) < len(Y):
        print('数据 X 有缺失，长度与数据 Y 不一致！！！')
    elif len(X) > len(Y):
        print('数据 Y 有缺失，长度与数据 X 不一致！！！')
    
    line = (
        Line(
            opts.InitOpts(
                bg_color = '#FFFFFF' ,
                width = '1333px' ,
                height = '600px'
            )
        )
        .add_xaxis(X)
        .add_yaxis(
            Y_name , 
            Y , 
            color = '#FF4500' ,
            symbol_size = [7 , 7] ,
            is_connect_nones = True ,
            markpoint_opts = opts.MarkPointOpts(
                data = [
                    opts.MarkPointItem(type_ = 'min' , name = 'Min') ,          # 标记 min
                    opts.MarkPointItem(type_ = 'max' , name = 'Max')            # 标记 max
                ] ,
                symbol_size = [45 , 45] , 
                label_opts = opts.LabelOpts(
                    formatter = '{b}' , 
                    position = 'inside'
                )
            ) ,
            markline_opts = opts.MarkLineOpts(
                symbol = ['circle' , 'triangle'] ,
                symbol_size = [11 , 11] ,
                label_opts = opts.LabelOpts(
                    font_weight = 'bold' , 
                    font_size = 16 , 
                    color = 'auto' , 
                    formatter = ' {c} 均值线'
                ) ,
                linestyle_opts = opts.LineStyleOpts(
                    width = 1.5 , 
                    type_ = 'dashed'
                ) , 
                data = [
                    opts.MarkLineItem(type_ = 'average' , name = Y_name)       # 标记 mean ，会忽略 None 计算 mean
                ]
            ) , 
            label_opts = opts.LabelOpts(is_show = True , font_weight='bold' , color = 'auto') ,
            linestyle_opts = opts.LineStyleOpts(width = 2)
        )
        .set_global_opts(
            datazoom_opts = opts.DataZoomOpts(           # 区域缩放滚动
                range_start = 0 , 
                range_end = 100
            ) ,
            yaxis_opts = opts.AxisOpts(
                axislabel_opts = opts.LabelOpts(
                    formatter = '{value} ' + Y_unit , 
                    font_weight = 'bold' , 
                    font_size = 14 , 
                    color = 'auto')
            ) , 
            xaxis_opts = opts.AxisOpts(
                axislabel_opts = opts.LabelOpts(
                    is_show = True ,
                    formatter = '{value}' , 
                    font_weight = 'bold' , 
                    font_size = 14 , 
                    color = 'auto' , 
                    rotate = 0)
            ) ,
            legend_opts = opts.LegendOpts(pos_top = 15) ,
            toolbox_opts = opts.ToolboxOpts(pos_top = 10 , pos_left = 950) ,
            tooltip_opts=opts.TooltipOpts(
                is_show = True , 
                trigger = 'axis' , 
                axis_pointer_type = 'cross')
        )
    )
    return line